# Generate the table

In our dataset, there's no data on the users, but we do have the tables of interactions between users and recipes (rating).  
We can generate a table of users by extracting the users from the interactions tables.  

The users table defined as follows:
```sql
-- User table
CREATE TABLE dbs.user (
    user_id INT PRIMARY KEY,
    first_name VARCHAR(45),
    last_name VARCHAR(45),
    date_of_birth DATE,
    weight TINYINT,
    height TINYINT,
    sex ENUM('M', 'F'),
    email VARCHAR(255) UNIQUE NOT NULL,
    password_hash VARCHAR(60) NOT NULL,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP
);
```

# 0. Importing the necessary libraries

In [2]:
!pip install pandas Faker tqdm bcrypt

In [1]:
import os
import ast
import json
import bcrypt
import numpy as np
import pandas as pd
from tqdm import tqdm

from faker import Faker

os.chdir(r"C:\Users\gilbe\Documents\Academics\M.Sc\Year 4\Semester A\DB Workshop")

In [7]:
path = "raw-data_interaction.csv"
df = pd.read_csv(path)
df.head()

,user_id,recipe_id,rating,dateLastModified
0,8542392,222388,5,2017-04-22T12:46:43.663\n
1,11174581,222388,5,2013-06-20T15:50:25.96\n
2,8262477,222388,5,2015-02-14T07:27:51.307\n
3,3574785,240488,5,2017-10-07T18:20:08.973\n
4,12145410,240488,2,2018-01-06T00:06:09.563\n


In [8]:
# pd.to_datetime(df["dateLastModified"].str[:-1], format="%Y-%m-%dT%H:%M:%S.%f")
df["datetime"] = pd.to_datetime(df["dateLastModified"].str[:-1], format="mixed")

In [9]:
df["datetime"].describe()

count                          3794003
mean     2010-10-22 14:10:57.267878656
min         2000-02-07 21:06:30.137000
25%         2008-02-25 06:23:30.880000
50%      2010-10-25 12:15:48.150000128
75%      2013-12-24 06:31:24.813499904
max         2018-03-16 07:19:52.260000
Name: datetime, dtype: object

In [10]:
# Let's move the dates forward, so the last modified date is today
offset = pd.to_datetime("today") - df["datetime"].max()
df["datetime"] = df["datetime"] + offset

In [11]:
df["datetime"].describe()

count                          3794003
mean     2017-08-03 18:33:08.130244864
min         2006-11-20 01:28:40.999366
25%      2014-12-07 10:45:41.742365952
50%      2017-08-06 16:37:59.012366080
75%      2020-10-05 10:53:35.675865856
max         2024-12-26 11:42:03.122366
Name: datetime, dtype: object

In [12]:
df["user_id"].value_counts().describe().to_frame().T

,count,mean,std,min,25%,50%,75%,max
count,1160267.0,3.26994,13.54242,1.0,1.0,1.0,2.0,4003.0


In [14]:
from typing import Optional, TypedDict

class User(TypedDict):
    user_id: int
    name: str
    date_of_birth: str
    weight: int
    height: int
    sex: str
    email: str
    password_hash: str
    created_at: str
    updated_at: Optional[str]

# 1. Simulate the user data

In [15]:
def generate_date_of_birth(created_at: pd.Series, mean_age: float = 70, std_age: float = 27, size: int = None) -> pd.Series:
    """Generate random dates of birth using normal distribution with mean age.
    
    Args:
        created_at: Series of user account creation dates
        mean_age: Mean age in years (default 70 based on data)
        std_age: Standard deviation of age in years (default 27 based on data)
        size: Number of dates to generate (default None, uses length of created_at)

    Returns:
        Series of birth dates as YYYY-MM-DD strings
    """
    if size is None:
        size = len(created_at)
        
    # Sample ages from normal distribution, ensuring minimum age of 18
    ages = np.maximum(18, np.random.normal(mean_age, std_age, size))
    
    # Vectorized calculation of birth dates
    birth_dates = created_at - pd.TimedeltaIndex(ages.astype(int) * 365.25, unit='D')
    
    return birth_dates.dt.strftime("%Y-%m-%d")

For each user, we will generate: name, email, password_hash, phone, address, date_of_birth, weight, height and sex


In [16]:
user_ids = df["user_id"].unique()
users = dict()
fake = Faker()
Faker.seed(1337)

password = "123456"
password_hash = bcrypt.hashpw(password.encode(), bcrypt.gensalt()).decode()

weight_params = {
    "M": {"mean": 80, "std": 10},
    "F": {"mean": 60, "std": 10},
}
height_params = {
    "M": {"mean": 180, "std": 10}, 
    "F": {"mean": 165, "std": 10},
}

In [17]:
# Pre-calculate first interaction dates for all users
first_interactions = df.groupby("user_id")["datetime"].min()

In [18]:
# Generate random days offset vector for all users at once
days_offset = np.random.randint(1, 31, size=len(user_ids))

# Generate sex for all users at once 
sexes = np.random.choice(["M", "F"], size=len(user_ids))

# Generate weights and heights vectors
weights = np.where(sexes == "M", 
                  np.random.normal(weight_params["M"]["mean"], weight_params["M"]["std"], len(user_ids)),
                  np.random.normal(weight_params["F"]["mean"], weight_params["F"]["std"], len(user_ids)))

heights = np.where(sexes == "M",
                  np.random.normal(height_params["M"]["mean"], height_params["M"]["std"], len(user_ids)), 
                  np.random.normal(height_params["F"]["mean"], height_params["F"]["std"], len(user_ids)))

In [19]:
# Pre-generate names and emails
names = [fake.name() for _ in tqdm(range(len(user_ids)))]

100%|██████████| 1160267/1160267 [02:12<00:00, 8777.19it/s]


In [24]:
dates_of_birth = generate_date_of_birth(created_at=first_interactions, mean_age=30, std_age=3, size=len(user_ids))

In [25]:
for i, user_id in enumerate(tqdm(user_ids)):
    created_at = first_interactions[user_id] - pd.Timedelta(days=days_offset[i])
    email = f"{user_id}@example.com"
    
    user = {
        "id": user_id,
        "name": names[i],
        "email": email, 
        "password_hash": password_hash,
        "date_of_birth": dates_of_birth.iloc[i],
        "weight": weights[i],
        "height": heights[i],
        "sex": sexes[i],
        "created_at": created_at.strftime("%Y-%m-%d"),
    }
    users[user_id] = user

  0%|          | 0/1160267 [00:00<?, ?it/s]

100%|██████████| 1160267/1160267 [00:54<00:00, 21282.85it/s]


In [26]:
users_int = {int(k): v for k, v in tqdm(users.items())}

100%|██████████| 1160267/1160267 [00:00<00:00, 1701135.27it/s]


In [35]:
with open("users.json", "w") as f:
    json.dump(users_int, f, indent=4)

In [4]:
with open("users.json", "r") as f:
    users_int = json.load(f)


In [36]:
users[list(users.keys())[0]]

{'name': 'Kim Diaz',
 'email': '8542392@example.com',
 'password_hash': '$2b$12$HLyLfbyxXovuTREeRlfUAODUuM8Rrf15VuFnaDbBBKgduzv/4MT5y',
 'date_of_birth': '1975-12-08',
 'weight': 70.71667777793547,
 'height': 156.48908775480072,
 'sex': 'F',
 'created_at': '2020-09-30'}

In [37]:
user_df = pd.DataFrame(users_int).T
user_df.index.name = "user_id"
user_df = user_df.reset_index()
user_df.head()

,user_id,name,email,password_hash,date_of_birth,weight,height,sex,created_at
0,8542392,Kim Diaz,8542392@example.com,$2b$12$HLyLfbyxXovuTREeRlfUAODUuM8Rrf15VuFnaDb...,1975-12-08,70.716678,156.489088,F,2020-09-30
1,11174581,Carol Holloway,11174581@example.com,$2b$12$HLyLfbyxXovuTREeRlfUAODUuM8Rrf15VuFnaDb...,1982-10-04,72.173259,160.737744,M,2019-12-12
2,8262477,Richard Dorsey,8262477@example.com,$2b$12$HLyLfbyxXovuTREeRlfUAODUuM8Rrf15VuFnaDb...,1974-11-20,58.747264,154.202618,F,2021-11-18
3,3574785,Willie Harris,3574785@example.com,$2b$12$HLyLfbyxXovuTREeRlfUAODUuM8Rrf15VuFnaDb...,1981-12-14,75.823579,171.805924,F,2021-11-24
4,12145410,Susan Rodriguez,12145410@example.com,$2b$12$HLyLfbyxXovuTREeRlfUAODUuM8Rrf15VuFnaDb...,1974-12-01,62.092151,187.15616,M,2022-09-29


In [38]:
user_df["weight"] = user_df["weight"].astype(int)
user_df["height"] = user_df["height"].astype(int)

In [39]:
(2025 - pd.to_datetime(user_df["date_of_birth"]).dt.year).describe().to_frame().T


,count,mean,std,min,25%,50%,75%,max
date_of_birth,1160267.0,36.938749,5.311992,19.0,33.0,37.0,40.0,60.0


In [40]:
user_df.to_csv("users.csv", index=False)

In [3]:
user_df = pd.read_csv("users.csv")
user_df.head()


,name,email,password_hash,date_of_birth,weight,height,sex,created_at
0,Kim Diaz,keith57@example.org,$2b$12$gQ5cTbFboXtm3B3ArLfru.65J3oVzCl7oZuYAH8...,1977-12-06,65,153,F,2020-09-29
1,Carol Holloway,paulhunter@example.org,$2b$12$gQ5cTbFboXtm3B3ArLfru.65J3oVzCl7oZuYAH8...,1981-10-03,108,181,M,2019-12-23
2,Richard Dorsey,harrishannah@example.com,$2b$12$gQ5cTbFboXtm3B3ArLfru.65J3oVzCl7oZuYAH8...,1981-11-19,81,169,M,2021-11-24
3,Willie Harris,meganmurray@example.com,$2b$12$gQ5cTbFboXtm3B3ArLfru.65J3oVzCl7oZuYAH8...,1976-12-13,82,158,M,2021-12-07
4,Susan Rodriguez,amy81@example.com,$2b$12$gQ5cTbFboXtm3B3ArLfru.65J3oVzCl7oZuYAH8...,1976-11-29,60,151,F,2022-10-24


# 2. Generate the other tables

## 2.1. load the processed data: nutritions, recipes and contains

In [181]:
nutritions_df = pd.read_csv("nutritions.csv")
nutritions_df.head()

,name,unit,average_daily_value
0,Carbohydrates,g,300.342400
1,Niacin Equivalents,mg,12.706413
2,Dietary Fiber,g,24.048217
3,Thiamin,mg,1.099229
4,Vitamin A - IU,IU,4834.502500


In [42]:
recipes_df = pd.read_csv("recipes.csv")
recipes_df.head()


,recipe_id,recipe_name,total_time,image,directions,ingredients
0,240488,"Pork Loin, Apples, and Sauerkraut",165,https://images.media-allrecipes.com/userphotos...,"""Prep\n15 m\nCook\n2 h 30 m\nReady In\n2 h 45...",sauerkraut drained^Granny Smith apples sliced^...
1,218939,Foolproof Rosemary Chicken Wings,60,https://images.media-allrecipes.com/userphotos...,"""Prep\n20 m\nCook\n40 m\nReady In\n1 h\nPrehe...",chicken wings^sprigs rosemary^head garlic^oliv...
2,87211,Chicken Pesto Paninis,20,https://images.media-allrecipes.com/userphotos...,"""Prep\n15 m\nCook\n5 m\nReady In\n20 m\nPrehe...",focaccia bread quartered^prepared basil pesto^...
3,245714,Potato Bacon Pizza,70,https://images.media-allrecipes.com/userphotos...,"""Prep\n20 m\nCook\n45 m\nReady In\n1 h 10 m\n...",red potatoes^strips bacon^Sauce:^heavy whippin...
4,218545,Latin-Inspired Spicy Cream Chicken Stew,505,https://images.media-allrecipes.com/userphotos...,"""Prep\n10 m\nCook\n8 h 15 m\nReady In\n8 h 25...",skinless boneless chicken breast halves^diced ...


In [43]:
contains_df = pd.read_csv("contains.csv")
contains_df.head()

,recipe_id,nutrition_name,value
0,240488,Vitamin C,52.768480
1,218939,Vitamin C,5.307448
2,87211,Vitamin C,18.015020
3,245714,Vitamin C,0.905797
4,218545,Vitamin C,14.064870


In [46]:
# validate that all recipe_ids in the contains table are in the recipes table
assert set(contains_df["recipe_id"].unique()) == set(recipes_df["recipe_id"].unique())


## 2.2. Create the rating table

In [47]:
interactions_df = pd.read_csv("raw-data_interaction.csv")
interactions_df.head()

,user_id,recipe_id,rating,dateLastModified
0,8542392,222388,5,2017-04-22T12:46:43.663\n
1,11174581,222388,5,2013-06-20T15:50:25.96\n
2,8262477,222388,5,2015-02-14T07:27:51.307\n
3,3574785,240488,5,2017-10-07T18:20:08.973\n
4,12145410,240488,2,2018-01-06T00:06:09.563\n


In [48]:
# validate all recipe_ids and user_ids are in the recipes and users tables
if not set(interactions_df["recipe_id"].unique()) == set(recipes_df["recipe_id"].unique()):
    print("Some recipe_ids are not in the recipes table")
    interactions_df = interactions_df[interactions_df["recipe_id"].isin(recipes_df["recipe_id"])]
if not set(interactions_df["user_id"].unique()) == set(user_df["user_id"].unique()):
    print("Some user_ids are not in the users table")
    interactions_df = interactions_df[interactions_df["user_id"].isin(user_df["user_id"])]

Some recipe_ids are not in the recipes table
Some user_ids are not in the users table


In [49]:
interactions_df["datetime"] = pd.to_datetime(interactions_df["dateLastModified"].str[:-1], format="mixed")
interactions_df["datetime"].describe()

count                          3781403
mean     2010-10-18 15:45:04.105940480
min         2000-02-07 21:06:30.137000
25%         2008-02-23 13:32:01.780000
50%      2010-10-22 11:18:20.049999872
75%      2013-12-17 15:44:19.816499968
max         2018-03-16 07:19:52.260000
Name: datetime, dtype: object

In [50]:
# Let's move the dates forward, so the last modified date is today
offset = pd.to_datetime("today") - interactions_df["datetime"].max()
interactions_df["datetime"] = interactions_df["datetime"] + offset
interactions_df["datetime"].describe()

count                          3781403
mean     2017-07-30 20:26:34.349565440
min         2006-11-20 01:48:00.380625
25%      2014-12-05 18:13:32.023624960
50%      2017-08-03 15:59:50.293625088
75%      2020-09-28 20:25:50.060124928
max         2024-12-26 12:01:22.503625
Name: datetime, dtype: object

In [51]:
interactions_df = interactions_df.drop(columns=["dateLastModified"])
interactions_df.rename(
    columns={
        "datetime": "created_at",
    },
    inplace=True,
)
interactions_df.to_csv("rating.csv", index=False)

In [52]:
rating_df = pd.read_csv("rating.csv")
del interactions_df
rating_df.head()

,user_id,recipe_id,rating,created_at
0,3574785,240488,5,2024-07-19 23:01:39.216625
1,12145410,240488,2,2024-10-18 04:47:39.806625
2,13549066,240488,5,2023-07-17 23:24:11.950625
3,3450379,240488,5,2022-07-30 17:11:07.840625
4,9823884,240488,5,2024-10-20 06:40:46.513625


## 2.3. Create the diet table


In [147]:
diets_keywords = {
    "vegan": [
        "chicken",
        "beef",
        "meat",
        "egg",
        "milk",
        "cheese",
        "yogurt",
        "cream",
        "butter",
        "fish",
        "seafood",
        "honey",
        "fish",
        "seafood",
        "gelatin",
        "lard",
        "bacon",
        "pork",
        "ham",
    ],
    "vegetarian": [
        "chicken",
        "beef",
        "meat",
        "fish",
        "seafood",
        "gelatin",
        "lard",
        "bacon",
        "pork",
        "ham",
    ],
    "gluten_free": [
        "wheat",
        "flour",
        "bread",
        "pasta",
        "barley",
        "rye",
        "oats",
        "breadcrumbs",
        "soy sauce",
    ],
    "dairy_free": [
        "milk",
        "cheese",
        "yogurt",
        "cream",
        "butter",
        "whey",
        "casein",
    ],
    "nut_allergy": [
        "peanuts",
        "almonds",
        "cashews",
        "walnuts",
        "pecans",
        "pistachios",
        "hazelnuts",
        "macadamia",
        "pine nuts",
    ],
    "shellfish_allergy": [
        "shrimp",
        "crab",
        "lobster",
        "clams",
        "mussels",
        "oysters",
        "scallops",
        "crawfish",
    ],
    "egg_allergy": ["egg", "mayonnaise", "meringue", "custard", "hollandaise", "aioli"],
}

In [175]:
diet_df = pd.DataFrame()
diet_df["name"] = diets_keywords.keys()
diet_df["keywords"] = diets_keywords.values()
diet_df["keywords"] = diet_df["keywords"].apply(lambda x: json.dumps(x))
diet_df = diet_df.reset_index().rename(columns={"index": "diet_id"})
diet_df["diet_id"] = diet_df["diet_id"] + 1
diet_df.to_csv("diet.csv", index=False)
diet_df = pd.read_csv("diet.csv")
diet_df["keywords"] = diet_df["keywords"].apply(json.loads)
diet_df.head()


,diet_id,name,keywords
0,1,vegan,"[chicken, beef, meat, egg, milk, cheese, yogur..."
1,2,vegetarian,"[chicken, beef, meat, fish, seafood, gelatin, ..."
2,3,gluten_free,"[wheat, flour, bread, pasta, barley, rye, oats..."
3,4,dairy_free,"[milk, cheese, yogurt, cream, butter, whey, ca..."
4,5,nut_allergy,"[peanuts, almonds, cashews, walnuts, pecans, p..."


## 2.4. Create the fits table


In [55]:
recipes_df.head()

,recipe_id,recipe_name,total_time,image,directions,ingredients
0,240488,"Pork Loin, Apples, and Sauerkraut",165,https://images.media-allrecipes.com/userphotos...,"""Prep\n15 m\nCook\n2 h 30 m\nReady In\n2 h 45...",sauerkraut drained^Granny Smith apples sliced^...
1,218939,Foolproof Rosemary Chicken Wings,60,https://images.media-allrecipes.com/userphotos...,"""Prep\n20 m\nCook\n40 m\nReady In\n1 h\nPrehe...",chicken wings^sprigs rosemary^head garlic^oliv...
2,87211,Chicken Pesto Paninis,20,https://images.media-allrecipes.com/userphotos...,"""Prep\n15 m\nCook\n5 m\nReady In\n20 m\nPrehe...",focaccia bread quartered^prepared basil pesto^...
3,245714,Potato Bacon Pizza,70,https://images.media-allrecipes.com/userphotos...,"""Prep\n20 m\nCook\n45 m\nReady In\n1 h 10 m\n...",red potatoes^strips bacon^Sauce:^heavy whippin...
4,218545,Latin-Inspired Spicy Cream Chicken Stew,505,https://images.media-allrecipes.com/userphotos...,"""Prep\n10 m\nCook\n8 h 15 m\nReady In\n8 h 25...",skinless boneless chicken breast halves^diced ...


In [59]:
recipe_diets = set()
for recipe_id, ingredients in zip(recipes_df["recipe_id"], recipes_df["ingredients"]):
    for i, (diet_id, diet_name, keywords) in diet_df.iterrows():
        if any(keyword in ingredients for keyword in ast.literal_eval(keywords)):
            continue
        recipe_diets.add((recipe_id, diet_id))
fits_df = pd.DataFrame(recipe_diets, columns=["recipe_id", "diet_id"])
fits_df.to_csv("fits.csv", index=False)
fits_df = pd.read_csv("fits.csv")
fits_df.head()


,recipe_id,diet_id
0,240418,5
1,240022,5
2,19484,0
3,17106,4
4,254947,1


## 2.5. Create the eats table

This table will be created by joining the rating table and the users table.
we'll set an assumption that every user rate 20-60% of the recipes he/she has seen.

So we have two steps:
1. Randomly select 20-60% of the recipes for each user
2. Create the eats table by joining the rating table and the users table

The eats table will have the following columns:
- user_id
- recipe_id
- created_at

In [61]:
# the east table based on the rating table and the users table
# if the user has rated the recipe, he/she has eaten it ~2-3 hours before the rating

eats_df = rating_df.copy()
eats_df["created_at"] = pd.to_datetime(eats_df["created_at"]) - pd.Timedelta(hours=2)
eats_df.drop(columns=["rating"], inplace=True)
eats_df.to_csv("eats.csv", index=False)
eats_df = pd.read_csv("eats.csv")
eats_df.head()


,user_id,recipe_id,created_at
0,3574785,240488,2024-07-19 21:01:39.216625
1,12145410,240488,2024-10-18 02:47:39.806625
2,13549066,240488,2023-07-17 21:24:11.950625
3,3450379,240488,2022-07-30 15:11:07.840625
4,9823884,240488,2024-10-20 04:40:46.513625


In [62]:
# now, for each recipe, we will sample between 1-5 different recipes that the user has eaten
from tqdm import tqdm
import numpy as np

# Get all unique users and recipes once
unique_users = eats_df["user_id"].unique()
all_recipes = np.array(list(recipes_df["recipe_id"].unique()))

# Create mapping of user_id to recipes eaten for faster lookup
user_recipes_map = eats_df.groupby("user_id")["recipe_id"].apply(set).to_dict()

# Create mapping of user_id to creation date for faster lookup
user_created_map = user_df.set_index("user_id")["created_at"].to_dict()

# Pre-calculate now timestamp
now = pd.Timestamp.now().timestamp()

In [63]:
# Pre-allocate list with estimated size
n_users = len(unique_users)
additional_eats_size = n_users * 3  # Average of 3 new recipes per user
additional_eats = []

# Convert to numpy arrays for faster operations
all_recipes_set = set(all_recipes)
now_ts = now

# For each user with progress bar
for user_id in tqdm(unique_users, desc="Processing users"):
    # Get recipes this user has already eaten from map
    user_recipes = user_recipes_map.get(user_id, set())
    
    # Get available recipes using set difference (faster than np.isin)
    available_recipes = list(all_recipes_set - user_recipes)
    
    if available_recipes:
        # Sample 1-5 new recipes for this user to eat
        n_new_recipes = min(np.random.randint(1, 6), len(available_recipes))
        new_recipes = np.random.choice(available_recipes, size=n_new_recipes, replace=False)
        
        # Get user creation time from map and convert to timestamp
        user_created = pd.to_datetime(user_created_map[user_id]).timestamp()
        
        # Generate timestamps
        timestamps = np.random.uniform(user_created, now_ts, size=n_new_recipes)
        
        # Extend list with new records
        additional_eats.extend({
            "user_id": user_id,
            "recipe_id": recipe_id,
            "created_at": pd.Timestamp.fromtimestamp(ts)
        } for recipe_id, ts in zip(new_recipes, timestamps))

# Create DataFrame from list of dicts
additional_eats_df = pd.DataFrame(additional_eats)

# Combine with existing data and sort
eats_df = pd.concat([eats_df, additional_eats_df], ignore_index=True)
eats_df.sort_values(["user_id", "created_at"], inplace=True)

# Save updated eats table
eats_df.to_csv("eats.csv", index=False)

Processing users: 100%|██████████| 1155839/1155839 [2:05:09<00:00, 153.92it/s] 


In [64]:
print(eats_df.shape)
eats_df.head()

(7248418, 3)


,user_id,recipe_id,created_at
4379153,16,23934,2013-07-11 00:26:22.453177
1720771,16,7375,2006-12-07 18:07:36.776625
1598973,16,8070,2006-12-22 12:54:53.133625
1140893,16,8887,2007-03-01 12:27:06.826625
3207175,16,11763,2008-02-24 19:42:17.146625


In [66]:
# validate that all recipe_ids in the eats table are in the recipes table
assert eats_df["recipe_id"].isin(set(recipes_df["recipe_id"].unique())).all()
# validate that all user_ids in the eats table are in the users table
assert eats_df["user_id"].isin(set(user_df["user_id"].unique())).all()


## 2.6. Create the follows table  

This table is the many-to-many relationship between users and diets.  
We'll populate this table by reviewing all the recipes that each user has eaten and then checking if the recipe fits any of the diets.


In [67]:
fits_df.head()

,recipe_id,diet_id
0,240418,5
1,240022,5
2,19484,0
3,17106,4
4,254947,1


In [68]:
# group the fits_df by recipe_id to get the list of diets for each recipe
diest_lists = fits_df.groupby("recipe_id")["diet_id"].apply(list).to_frame()
diest_lists.head()


,diet_id
recipe_id,
6663,"[4, 5, 1, 2]"
6664,"[4, 5, 1]"
6665,"[4, 5, 1, 3]"
6666,"[5, 3, 1]"
6667,"[6, 4, 5, 1]"


In [69]:
enriched_recipes_df = pd.merge(recipes_df, diest_lists.reset_index(), how="left", left_on="recipe_id", right_on="recipe_id")
enriched_recipes_df.head()


,recipe_id,recipe_name,total_time,image,directions,ingredients,diet_id
0,240488,"Pork Loin, Apples, and Sauerkraut",165,https://images.media-allrecipes.com/userphotos...,"""Prep\n15 m\nCook\n2 h 30 m\nReady In\n2 h 45...",sauerkraut drained^Granny Smith apples sliced^...,"[4, 5, 6, 2, 3]"
1,218939,Foolproof Rosemary Chicken Wings,60,https://images.media-allrecipes.com/userphotos...,"""Prep\n20 m\nCook\n40 m\nReady In\n1 h\nPrehe...",chicken wings^sprigs rosemary^head garlic^oliv...,"[3, 4, 2, 5, 6]"
2,87211,Chicken Pesto Paninis,20,https://images.media-allrecipes.com/userphotos...,"""Prep\n15 m\nCook\n5 m\nReady In\n20 m\nPrehe...",focaccia bread quartered^prepared basil pesto^...,"[5, 6, 4]"
3,245714,Potato Bacon Pizza,70,https://images.media-allrecipes.com/userphotos...,"""Prep\n20 m\nCook\n45 m\nReady In\n1 h 10 m\n...",red potatoes^strips bacon^Sauce:^heavy whippin...,"[4, 5, 6]"
4,218545,Latin-Inspired Spicy Cream Chicken Stew,505,https://images.media-allrecipes.com/userphotos...,"""Prep\n10 m\nCook\n8 h 15 m\nReady In\n8 h 25...",skinless boneless chicken breast halves^diced ...,"[2, 6, 4, 5]"


In [70]:
enriched_eats_df = pd.merge(eats_df, enriched_recipes_df, how="left", left_on="recipe_id", right_on="recipe_id")
enriched_eats_df.head()

,user_id,recipe_id,created_at,recipe_name,total_time,image,directions,ingredients,diet_id
0,16,23934,2013-07-11 00:26:22.453177,Hot Onion Dip,40,http://images.media-allrecipes.com/userphotos/...,"""Prep\n10 m\nCook\n30 m\nReady In\n40 m\nPreh...",cream cheese^onion^grated Parmesan cheese^mayo...,"[1, 4, 5, 2]"
1,16,7375,2006-12-07 18:07:36.776625,Black Russian Cake I,60,http://images.media-allrecipes.com/userphotos/...,"""Prep\n20 m\nCook\n40 m\nReady In\n1 h\nPrehe...",moist yellow cake mix^instant chocolate puddin...,"[5, 3, 4, 1, 2]"
2,16,8070,2006-12-22 12:54:53.133625,Blackberry Jam Cake I,0,http://images.media-allrecipes.com/userphotos/...,"""Combine flour, baking powder, salt, cocoa, c...",butter^white sugar^egg yolks^baking soda^butte...,"[1, 4, 5]"
3,16,8887,2007-03-01 12:27:06.826625,Chicken Marsala,30,https://images.media-allrecipes.com/userphotos...,"""Prep\n10 m\nCook\n20 m\nReady In\n30 m\nIn a...",all-purpose flour for coating^salt^ground blac...,"[4, 5, 6]"
4,16,11763,2008-02-24 19:42:17.146625,Seafood Lasagna I,0,https://images.media-allrecipes.com/userphotos...,"""Cook the lasagna noodles in a large pot of b...",onion^butter^cottage cheese^cream cheese^dried...,"[1, 2, 4]"


In [71]:
# get the list of diets for each user
user_diet_eats = enriched_eats_df[enriched_eats_df["diet_id"].notna()].groupby("user_id")["diet_id"].apply(list).to_frame()
user_diet_eats.head()


,diet_id
user_id,
16,"[[1, 4, 5, 2], [5, 3, 4, 1, 2], [1, 4, 5], [4,..."
21,"[[4, 1, 2, 5, 6], [5, 6, 2, 4], [4, 1, 5], [4,..."
22,"[[4, 5, 1, 2, 6], [2, 3, 0, 1, 5, 6], [1, 5, 6..."
26,"[[4, 5, 6], [5, 2, 6, 3, 4], [4, 5, 2, 6], [4,..."
32,"[[4, 5, 2, 1], [2, 3, 4, 5, 6], [5, 3, 1], [2,..."


In [72]:
# Now let's create the follows table. If a user choose all of his recipes by specific diets, we will create a follows record for each diet.
def count_diet_eats(diet_ids_lists):
    if not diet_ids_lists:
        return set()
    follows = set()
    all_diets = set(diet_df["diet_id"])
    for diet_id in all_diets:
        if all(diet_id in diet_ids_list for diet_ids_list in diet_ids_lists):
            follows.add(diet_id)
    return follows

user_diet_eats["follows"] = user_diet_eats["diet_id"].apply(count_diet_eats)
user_diet_eats.head()

,diet_id,follows
user_id,,
16,"[[1, 4, 5, 2], [5, 3, 4, 1, 2], [1, 4, 5], [4,...",{}
21,"[[4, 1, 2, 5, 6], [5, 6, 2, 4], [4, 1, 5], [4,...",{5}
22,"[[4, 5, 1, 2, 6], [2, 3, 0, 1, 5, 6], [1, 5, 6...",{}
26,"[[4, 5, 6], [5, 2, 6, 3, 4], [4, 5, 2, 6], [4,...","{4, 5}"
32,"[[4, 5, 2, 1], [2, 3, 4, 5, 6], [5, 3, 1], [2,...",{}


In [73]:
user_diet_eats["follows"].apply(len).describe().to_frame().T


,count,mean,std,min,25%,50%,75%,max
follows,1155839.0,1.872713,1.035617,0.0,1.0,2.0,2.0,7.0


In [74]:
user_diet_eats.head().reset_index()[["user_id", "follows"]].explode("follows").dropna()

,user_id,follows
1,21,5
3,26,4
3,26,5


In [75]:
# Now create the follows table
follows_df = user_diet_eats.reset_index()[["user_id", "follows"]].explode("follows")
follows_df.rename(columns={"follows": "diet_id"}, inplace=True)
follows_df.dropna(inplace=True)
follows_df.to_csv("follows.csv", index=False)
follows_df = pd.read_csv("follows.csv")
follows_df.head()

,user_id,diet_id
0,21,5
1,26,4
2,26,5
3,44,5
4,55,4


## 2.7. Create the admin table


In [80]:
ADMIN_IDS = [26, 21, 9521845]
user_df[user_df["user_id"].isin(ADMIN_IDS)]


,user_id,name,email,password_hash,date_of_birth,weight,height,sex,created_at
314120,21,Mark Rosario,21@example.com,$2b$12$HLyLfbyxXovuTREeRlfUAODUuM8Rrf15VuFnaDb...,1984-08-31,48,169,F,2009-09-15
511218,9521845,Melissa Cruz,9521845@example.com,$2b$12$HLyLfbyxXovuTREeRlfUAODUuM8Rrf15VuFnaDb...,1991-09-28,84,186,M,2019-03-30
533083,26,Meredith Warren,26@example.com,$2b$12$HLyLfbyxXovuTREeRlfUAODUuM8Rrf15VuFnaDb...,1990-01-20,81,166,F,2006-12-01


In [81]:
# Now create the admin table - columns are user_id, date_promoted (today)
admin_df = pd.DataFrame(
    {"user_id": ADMIN_IDS, "promoted_at": pd.Timestamp.now().strftime("%Y-%m-%d")}
)
admin_df.to_csv("admin.csv", index=False)
admin_df = pd.read_csv("admin.csv")
admin_df.head()

,user_id,promoted_at
0,26,2024-12-26
1,21,2024-12-26
2,9521845,2024-12-26


## 2.8. Create the user_nutrition table (tracks table)


In [82]:
daily_nutrition_df = pd.read_csv("daily_nutrition_values.csv")
daily_nutrition_df.head()

,Age Group,Sex,Nutrition Name,Unit,Recommended Daily Value
0,14-18,M,Dietary Fiber,g,28.0
1,14-18,F,Dietary Fiber,g,25.0
2,19-30,M,Dietary Fiber,g,34.0
3,19-30,F,Dietary Fiber,g,28.0
4,31-50,M,Dietary Fiber,g,31.0


In [83]:
def get_age_group(date_of_birth: str) -> str:
    age = 2025 - pd.to_datetime(date_of_birth).year
    if age < 18:
        return "14-18"
    elif age < 31:
        return "19-30"
    elif age < 51:
        return "31-50"
    else:
        return "51+"

In [205]:
age_groups = user_df["date_of_birth"].apply(get_age_group)
age_groups.value_counts()


date_of_birth
31-50    1026845
19-30     125696
51+         7726
Name: count, dtype: int64

In [201]:
all_nutritions = nutritions_df["name"].unique()
all_nutritions


array(['Carbohydrates', 'Niacin Equivalents', 'Dietary Fiber', 'Thiamin',
       'Vitamin A - IU', 'Calories', 'Fat', 'Saturated Fat', 'Potassium',
       'Calcium', 'Protein', 'Sodium', 'Vitamin C', 'Sugars', 'Folate',
       'Calories from Fat', 'Iron', 'Magnesium', 'Vitamin B6',
       'Cholesterol'], dtype=object)

In [210]:
tracks = set()
for user_id, age_group in tqdm(zip(user_df["user_id"], age_groups), desc="Processing users", total=len(age_groups)):
    user_sex = user_df[user_df["user_id"] == user_id]["sex"].values[0]
    user_nutritions = daily_nutrition_df[(daily_nutrition_df["Age Group"] == age_group) & (daily_nutrition_df["Sex"] == user_sex)]
    for nutrition_name, unit, average_daily_value in zip(user_nutritions["Nutrition Name"], user_nutritions["Unit"], user_nutritions["Recommended Daily Value"]):
        tracks.add((user_id, nutrition_name, average_daily_value))

user_nutrition_df = pd.DataFrame(tracks, columns=["user_id", "nutrition_name", "tracked_value"])
user_nutrition_df.to_csv("user_nutrition.csv", index=False)
user_nutrition_df = pd.read_csv("user_nutrition.csv")
user_nutrition_df.head()


Processing users: 100%|██████████| 1160267/1160267 [1:01:56<00:00, 312.19it/s]


,user_id,nutrition_name,tracked_value
0,2910440,Vitamin B6,1.3
1,14304347,Carbohydrates,130.0
2,16309484,Sodium,2300.0
3,18224627,Vitamin C,75.0
4,12273679,Carbohydrates,130.0


In [211]:
user_nutrition_df.shape

(23205340, 3)

In [ ]:
user_nutrition_df.shape

(23205340, 3)

In [212]:
user_nutrition_df["user_id"].isin(user_df["user_id"]).value_counts()


user_id
True    23205340
Name: count, dtype: int64

In [213]:
# verify that all user_ids in the user_nutrition table are in the users table
assert user_nutrition_df["user_id"].isin(set(user_df["user_id"].unique())).all()
assert user_nutrition_df["nutrition_name"].isin(set(nutritions_df["name"].unique())).all()



# 3. Create the database

In [88]:
import mysql.connector

In [214]:
mysql_user = "root"
mysql_password = "root"
mysql_host = "localhost"
mysql_port = 3306
mysql_database = "dbs"

# Create the database
conn = mysql.connector.connect(
    host=mysql_host,
    port=mysql_port,
    user=mysql_user,
    password=mysql_password,
    database=mysql_database,
    time_zone='+00:00'
)

cur = conn.cursor()
# test the connection
cur.execute("SHOW DATABASES")
print(cur.fetchall())

[('dbs',), ('information_schema',), ('mysql',), ('performance_schema',), ('sakila',), ('sys',), ('world',)]


In [215]:
# list the tables
cur.execute("SHOW TABLES")
print(cur.fetchall())

[('admin',), ('contains',), ('diet',), ('eats',), ('fits',), ('nutrition',), ('rating',), ('recipe',), ('user',), ('user_diet',), ('user_nutrition',)]


In [216]:
table_names_to_df = {
    "admin": admin_df,
    "contains": contains_df,
    "diet": diet_df,
    "eats": eats_df,
    "fits": fits_df,
    "nutrition": nutritions_df,
    "rating": rating_df,
    "recipe": recipes_df,
    "user": user_df,
    "user_diet": follows_df,
    "user_nutrition": user_nutrition_df,
}

In [192]:
# First, let's validate the columns of the tables
for table_name, df in table_names_to_df.items():
    columns = ", ".join([f"`{col}`" for col in df.columns])
    query = f"SHOW COLUMNS FROM {table_name}"
    cur.execute(query)
    db_columns = [col[0] for col in cur.fetchall()]
    for col in df.columns:
        if col not in db_columns:
            raise ValueError(f"Column {col} not found in database: {table_name}")
    


In [330]:
# we need to start with the user and recipe tables
tables_order = ["user", "recipe", "admin"]
for table_name in tables_order:
    df = table_names_to_df[table_name]
    for i, row in tqdm(df.iterrows(), desc=f"Inserting {table_name} rows", total=len(df)):
        columns = ", ".join([f"`{col}`" for col in df.columns])
        values = ", ".join([f"'{val}'" for val in row])
        query = f"INSERT INTO {table_name} ({columns}) VALUES ({values})"
        try:
            cur.execute(query)
            # Flush every 5000 rows
            if i > 0 and i % 5000 == 0:
                conn.commit()
        except Exception as e:
            print(f"Error inserting row {i} into {table_name}: {e}")
            print(f"Query: {query}")
            print(f"Row: {row}")
            raise e
    
    # Final commit for any remaining rows
    conn.commit()


Now, let's insert the rest of the tables

In [196]:
conn.commit()

In [217]:
tables_order = [
    # "rating",
    # "diet",
    # "nutrition",
    # "contains",
    # "eats",
    # "fits",
    # "user_diet",
    "user_nutrition",
]
for table_name in tables_order:
    df = table_names_to_df[table_name]
    for i, row in tqdm(
        df.iterrows(), desc=f"Inserting {table_name} rows", total=len(df)
    ):
        columns = ", ".join([f"`{col}`" for col in df.columns])
        values = []

        for val in row:
            if isinstance(val, list):
                json_str = json.dumps(val)
                values.append(f"%s")
            else:
                values.append(f"%s")

        values_str = ", ".join(values)
        query = f"INSERT INTO {table_name} ({columns}) VALUES ({values_str})"
        try:
            cur.execute(
                query,
                tuple(
                    str(val) if not isinstance(val, list) else json.dumps(val)
                    for val in row
                ),
            )
            # Flush every 5000 rows
            if i > 0 and i % 5000 == 0:
                conn.commit()
        except Exception as e:
            print(f"Error inserting row {i} into {table_name}: {e}")
            print(f"Query: {query}")
            print(f"Row: {row}")
            raise e

    # Final commit for any remaining rows
    conn.commit()

Inserting user_nutrition rows:   7%|▋         | 1554450/23205340 [29:19<11:45:54, 511.18it/s]

In [224]:
for table_name, df in table_names_to_df.items():
    for i, row in df.iterrows():
        columns = ", ".join([f"`{col}`" for col in df.columns])
        values = ", ".join([f"'{val}'" for val in row])
        query = f"INSERT INTO {table_name} ({columns}) VALUES ({values})"
        cur.execute(query)



ProgrammingError: 1054 (42S22): Unknown column 'date_promoted' in 'field list'